In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


## Video transcription
@forusone (shins777@gmail.com)

### Video transcription and translation

This colab used YT videos to test Gemini's transciption and translation.

### Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    google-cloud-speech \
    google-cloud-storage \
    google-cloud-videointelligence

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.7 MB/s eta 0:00:00


In [10]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik


Updated property [core/project].


### Initial set up

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
#MODEL = "gemini-1.5-pro-002"
MODEL = "gemini-pro-experimental"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

## Transcript and translation

### Helper functions

In [5]:

def generate(prompt:str,
             video_uri:str,
             response_schema)->str:
  """
  Function to call Gemini for Transcription and Translation.

  Args:
    prompt: Prompt to instruct transcription and translation.
    yt_uri: Youtube video URI to analyze.

  Returns:
    A transcription or translation with specified response schema.(JSON)

  """

  from vertexai.generative_models import (
        Part,
        SafetySetting,
        GenerationConfig
  )

  # Configuration to contorl LLM generation.
  generation_config = GenerationConfig(
      max_output_tokens = 8192,
      temperature=1,
      top_p =0.95,
      response_mime_type="application/json",
      response_schema=response_schema
  )

  # Sate setting
  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
  ]

  video1 = Part.from_uri(
      mime_type="video/*",
      uri=video_uri,
  )

  responses = model.generate_content(
      [prompt, video1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )

  return responses.text



### YT Videos to transcribe and translate

In [14]:

if False: # Videos in YT platform

  # video_uri="https://www.youtube.com/watch?v=PNTCM7cbrsc" # Small Talk
  # video_uri="https://www.youtube.com/watch?v=wrh-4GZN3aE" # Dad Won't Let Daughter Marry For Love
  # video_uri = "https://www.youtube.com/watch?v=CYyUuIXzGgI" # Game Theory Scene | 21(2008)
  video_uri = "https://www.youtube.com/watch?v=OoUVSHDbAeM" # Stephen Hawking Discovers The Black Hole Theory

else: # videos in GCS
  # video_uri="gs://tests_nov25_2024/translation/small_talk.mp4"  # Small Talk
  video_uri="gs://tests_nov25_2024/translation/Stephen Hawking Discovers The Black Hole Theory | The Theory Of Everything (2014) | Screen Bites.mp4" # Stephen Hawking Discovers The Black Hole Theory




### Video transcription

In [7]:
import json

prompt_transcript = """
당신을 비디오를 분석해서 transcript를 작성해야 하는 AI Assistant 입니다.
아래 가이드라인에 맞게 transcription을 작성해주세요.

1. 첨부된 비디오를 분석하여 아래와 같은 포맷으로 모든 대화 내용을 빠짐없이 출력해주세요.
2. 결과 출력 단위는 비디오 내의 장면이 구분되는 특정 장소를 기준으로 나누어서 출력해주세요.
3. 목소리를 기반으로 화자(speaker)를 정확하게 분리해서 영어로 출력해주세요.
4. 목소리외에 다양한 효과음, 감정표현은 괄호를 사용해서 반드시 자세히 표현해주세요.

"""

# Response schema can control the output of generation from Gemini.
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "location": { "type": "STRING",},
            "start_time": { "type": "STRING",},
            "end_time": { "type": "STRING",},
            "elapsed_time": { "type": "STRING",},
            "transcription": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "speaker": { "type": "STRING",},
                    "transcript": { "type": "STRING",},
                  }
                }
            },
        },
        "required": ["start_time","end_time","elapsed_time"],
    },
}

json_str = generate(prompt_transcript, video_uri, response_schema)
json_transcript = json.loads(json_str)


In [8]:
for transcription in json_transcript:
  print(transcription)
  print("-"*20)

{'location': 'Train Station', 'start_time': '00:00:05', 'end_time': '00:00:26', 'elapsed_time': '00:00:21', 'transcription': [{'speaker': 'Man 1', 'transcript': 'Come on, Stephen.'}, {'speaker': 'Man 2', 'transcript': 'Gotta move on.'}, {'speaker': 'Man 1', 'transcript': "What's wrong with you, man?\nChop chop."}, {'speaker': 'Train Staff', 'transcript': 'All aboard.'}]}
--------------------
{'location': 'Classroom', 'start_time': '00:00:27', 'end_time': '00:01:27', 'elapsed_time': '00:01:00', 'transcription': [{'speaker': 'Professor', 'transcript': 'A star.\nMore than three times the size of our sun ought to end its life how?\nWith a collapse.\nThe gravitational forces of the entire mass overcoming the electromagnetic forces of individual atoms, and so collapsing inwards.\nIf the star is massive enough, it will continue this collapse, creating a black hole.\nWhere the warping of space-time is so great that nothing can escape, not even light. It gets\nsmaller.\nSmaller.\nThe star, in f

### Video Translation

In [15]:

prompt_translation = f"""
당신을 원본 언어로 된 문서를 기반으로 비디오를 분석해서 한국어로 번역을 해야 하는 AI Assistant 입니다.
반드시 비디오의 모든 장면에 나타난 모든 대화 내용을 아래 가이드라인에 맞게 정확하게 번역 해주세요.

1. 번역을 할 때는 첨부된 json 내에서 transcription 부분만 해주세요.
2. 번역결과는 할 때는 json구조를 그대로 유지해서 표현해주세요.
3. 상황에 맞게 한국말의 존대말을 쓰거나 반말을 쓰도록 형태로 나타내 주세요.
4. 목소리외에 다양한 효과음, 감정표현은 괄호를 사용해서 반드시 자세히 표현해주세요.

원본 문서 : {json_transcript}

"""

json_str = generate(prompt_translation,video_uri, response_schema) # Used same response schema with transcription process.
json_translation = json.loads(json_str)


In [16]:
for transcription, translation in zip( json_transcript, json_translation):
  print(transcription)
  print(translation)
  print("-"*20)


{'location': 'Train Station', 'start_time': '00:00:05', 'end_time': '00:00:26', 'elapsed_time': '00:00:21', 'transcription': [{'speaker': 'Man 1', 'transcript': 'Come on, Stephen.'}, {'speaker': 'Man 2', 'transcript': 'Gotta move on.'}, {'speaker': 'Man 1', 'transcript': "What's wrong with you, man?\nChop chop."}, {'speaker': 'Train Staff', 'transcript': 'All aboard.'}]}
{'location': 'Train Station', 'start_time': '00:00:05', 'end_time': '00:00:26', 'elapsed_time': '00:00:21', 'transcription': [{'speaker': 'Man 1', 'transcript': '스티븐, 어서 타!'}, {'speaker': 'Man 2', 'transcript': '가야 해.'}, {'speaker': 'Man 1', 'transcript': '뭐 하는 거야, 친구?\n빨리 빨리!'}, {'speaker': 'Train Staff', 'transcript': '모두 탑승하세요.'}]}
--------------------
{'location': 'Classroom', 'start_time': '00:00:27', 'end_time': '00:01:27', 'elapsed_time': '00:01:00', 'transcription': [{'speaker': 'Professor', 'transcript': 'A star.\nMore than three times the size of our sun ought to end its life how?\nWith a collapse.\nThe gravi

### Compare transcript and translation

In [17]:
from IPython.display import display, Markdown
import pandas as pd

df = pd.DataFrame()

for transcript, translation in zip( json_transcript, json_translation):

  for script, trans in zip(transcript['transcription'], translation['transcription']):
    script_df = pd.DataFrame.from_dict(script, orient='index').T
    trans_df = pd.DataFrame.from_dict(trans, orient='index').T

    script_df['translation'] = trans_df['transcript']

    df = pd.concat([df, script_df], ignore_index=True)  # Concatenate script_df to df

df

,speaker,transcript,translation
0,Man 1,"Come on, Stephen.","스티븐, 어서 타!"
1,Man 2,Gotta move on.,가야 해.
2,Man 1,"What's wrong with you, man?\nChop chop.","뭐 하는 거야, 친구?\n빨리 빨리!"
3,Train Staff,All aboard.,모두 탑승하세요.
4,Professor,A star.\nMore than three times the size of our...,별.\n태양보다 세 배 이상 큰 별은 어떻게 생을 마감할까요?\n붕괴로 끝납니다.\...
5,Stephen,Space and time come to a stop.,공간과 시간이 멈추는군요.
6,Stephen,I wonder what would happen if you applied Penr...,블랙홀에 대한 펜로즈의 정리를 전체 우주에 적용하면 어떻게 될지 궁금하네.
7,Stephen,"If Einstein is right, right if general relativ...","아인슈타인이 맞다면, 일반 상대성 이론이 맞다면 우주는 팽창하고 있지, 그렇지?"
8,Jane,Yeah.,응.
9,Stephen,"Okay. So, if you reverse time,\nthen the unive...",좋아. 그럼 시간을 거꾸로 돌리면\n우주가 점점 작아지지.\n알겠어?\n그러니까\n...


## Validation translation

### Helper function for validation

### Prompt to recheck the previous translation

In [18]:
prompt_check = f"""
당신을 비디오를 분석해서 기존에 번역된 문서(translated document)를 참조하여 한번 더 정확하게 한국어로 번역을 해야 하는 AI Assistant 입니다.
반드시 비디오의 모든 장면에 나타난 모든 대화 내용을 아래 가이드라인에 맞게 정확하게 번역 해주세요.

1. 번역을 할 때는 첨부된 json 내에서 transcription 부분만 해주세요.
2. 상황에 맞게 한국말의 존대말을 쓰거나 반말을 쓰도록 형태로 나타내 주세요.
3. 목소리외에 다양한 효과음, 감정표현은 괄호를 사용해서 반드시 자세히 표현해주세요.
4. 기존 번역내용은 'original_translation' 에 표현해주고, 변경된 내용은 'checked_translation' 에 표현해주세요.

translated document : {json_translation}

"""

# Response schema can control the output of generation from Gemini.
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "location": { "type": "STRING",},
            "start_time": { "type": "STRING",},
            "end_time": { "type": "STRING",},
            "elapsed_time": { "type": "STRING",},
            "transcription": {
                "type": "ARRAY",
                "items" : {
                  "type": "OBJECT",
                  "properties": {
                    "speaker": { "type": "STRING",},
                    "original_translation": { "type": "STRING",},  # original translation
                    "checked_translation": { "type": "STRING",},   # recheck translation
                  }
                }
            },
        },
        "required": ["start_time","end_time","elapsed_time"],
    },
}

json_checked_translation_str = generate(prompt_check, video_uri, response_schema)
json_checked_translation = json.loads(json_checked_translation_str)


In [19]:
for checked_translation in json_checked_translation:
  print(checked_translation)
  print("-"*20)


{'location': 'Train Station', 'start_time': '00:00:05', 'end_time': '00:00:26', 'elapsed_time': '00:00:21', 'transcription': [{'speaker': 'Man 1', 'original_translation': '스티븐, 어서 타!', 'checked_translation': '스티븐, 어서 타!'}, {'speaker': 'Man 2', 'original_translation': '가야 해.', 'checked_translation': '가야 해.'}, {'speaker': 'Man 1', 'original_translation': '뭐 하는 거야, 친구?\n빨리 빨리!', 'checked_translation': '뭐 하는 거야, 친구?\n빨리 빨리!'}, {'speaker': 'Train Staff', 'original_translation': '모두 탑승하세요.', 'checked_translation': '모두 탑승하세요.'}]}
--------------------
{'location': 'Classroom', 'start_time': '00:00:27', 'end_time': '00:01:27', 'elapsed_time': '00:01:00', 'transcription': [{'speaker': 'Professor', 'original_translation': '별.\n태양보다 세 배 이상 큰 별은 어떻게 생을 마감할까요?\n붕괴로 끝납니다.\n전체 질량의 중력이 개별 원자의 전자기력을 극복하고 안쪽으로 붕괴됩니다.\n별의 질량이 충분히 크면 이 붕괴가 계속되어 블랙홀이 생성됩니다.\n시공간의 왜곡이 너무 커서 빛조차도 아무것도 빠져나갈 수 없습니다. 그것은 더\n작아집니다.\n더\n작아집니다.\n사실, 별은 원자, 심지어 아원자 입자가 말 그대로 점점 더 작은 공간으로 압축되면서 더 밀도가 높아집니다.\n그리고 마지막에는\n무엇이 남을까요?\n시공간

In [20]:
from IPython.display import display, Markdown
import pandas as pd

df2 = pd.DataFrame()

for transcript, translation in zip( json_transcript, json_checked_translation):

  for script, trans in zip(transcript['transcription'], translation['transcription']):
    script_df = pd.DataFrame.from_dict(script, orient='index').T
    trans_df = pd.DataFrame.from_dict(trans, orient='index').T

    script_df['org_translation'] = trans_df['original_translation']
    script_df['chk_translation'] = trans_df['checked_translation']

    df2 = pd.concat([df2, script_df], ignore_index=True)


df2

,speaker,transcript,org_translation,chk_translation
0,Man 1,"Come on, Stephen.","스티븐, 어서 타!","스티븐, 어서 타!"
1,Man 2,Gotta move on.,가야 해.,가야 해.
2,Man 1,"What's wrong with you, man?\nChop chop.","뭐 하는 거야, 친구?\n빨리 빨리!","뭐 하는 거야, 친구?\n빨리 빨리!"
3,Train Staff,All aboard.,모두 탑승하세요.,모두 탑승하세요.
4,Professor,A star.\nMore than three times the size of our...,별.\n태양보다 세 배 이상 큰 별은 어떻게 생을 마감할까요?\n붕괴로 끝납니다.\...,별.\n태양보다 세 배 이상 큰 별은 어떻게 생을 마감할까요?\n붕괴로 끝납니다.\...
5,Stephen,Space and time come to a stop.,공간과 시간이 멈추는군요.,공간과 시간이 멈추는군요.
6,Stephen,I wonder what would happen if you applied Penr...,블랙홀에 대한 펜로즈의 정리를 전체 우주에 적용하면 어떻게 될지 궁금하네.,블랙홀에 대한 펜로즈의 정리를 전체 우주에 적용하면 어떻게 될지 궁금하네.
7,Stephen,"If Einstein is right, right if general relativ...","아인슈타인이 맞다면, 일반 상대성 이론이 맞다면 우주는 팽창하고 있지, 그렇지?","아인슈타인이 맞다면, 일반 상대성 이론이 맞다면 우주는 팽창하고 있지, 그렇지?"
8,Jane,Yeah.,응.,응.
9,Stephen,"Okay. So, if you reverse time,\nthen the unive...",좋아. 그럼 시간을 거꾸로 돌리면\n우주가 점점 작아지지.\n알겠어?\n그러니까\n...,좋아. 그럼 시간을 거꾸로 돌리면\n우주가 점점 작아지지.\n알겠어?\n그러니까\n...
